In [30]:
#Standalone lines, delete table to re-perform import, close connection when needed, free memory
import gc
# c.execute("Drop table if exists Businesses")
# c.execute("Drop table if exists Checkins")
# c.execute("Drop table if exists Users")
# c.execute("Drop table if exists Tips")
# c.execute("Drop table if exists Reviews")
# conn.close()
gc.collect()

20

In [31]:
%%time
import pandas as pd
import json
import sqlite3
from sqlite3 import Error

def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            return conn
            
conn = create_connection(r"C:\Users\pajuc\Documents\yelp\data\yelp.db")
c = conn.cursor()

2.6.0
Wall time: 32 ms


In [32]:
%%time
import pandas as pd
prefix = "C:\\Users\\pajuc\\Documents\\yelp\\data\\"
filePaths = {
    "reviews": prefix + 'yelp_academic_dataset_review.json',
    "businesses": prefix + 'yelp_academic_dataset_business.json',
    "checkins": prefix + 'yelp_academic_dataset_checkin.json',
    "tips": prefix + 'yelp_academic_dataset_tip.json',
    "users": prefix + 'yelp_academic_dataset_user.json'
}

Wall time: 0 ns


In [33]:
%%time
#Loads business table, entire JSON fits in memory
dfBusinesses = pd.read_json(
    filePaths["businesses"], 
    lines=True
)

# This is a little brute force to get these dicts into SQL text. There are wrappers to maintain the struct if needed...
# Right now they will just be a large string. Since they are structured it may be easier to make some functions to
# extract info. TBD

dfBusinesses['attributes'] = dfBusinesses['attributes'].apply(json.dumps)
dfBusinesses['hours'] = dfBusinesses['hours'].apply(json.dumps)
dfBusinesses.to_sql('Businesses', con=conn)

Wall time: 10.2 s


In [34]:
%%time
#Loads checkins table
dfCheckins = pd.read_json(
    filePaths["checkins"], 
    lines=True
)
dfCheckins.to_sql('Checkins', con=conn)

Wall time: 8.99 s


In [35]:
%%time
#Loads tips table
dfTips = pd.read_json(
    filePaths["tips"], 
    lines=True
)
dfTips.to_sql('Tips', con=conn)

Wall time: 24 s


In [36]:
%%time
# Loads users table
# There is a bug in pandas 1.1.3 (et al) requiring nrows to be greater than # lines in file
# This was causing issues with chunking, but excess nrows fixed it.
# I read it was solved in 1.1.4 but didn't bother trying

for df in pd.read_json(
    filePaths["users"], 
    lines=True,
    orient='records',
    chunksize=100000,
    nrows=234236523
):
    new_users = pd.DataFrame(df)
    new_users.to_sql('Users', con=conn, if_exists='append')

Wall time: 2min 25s


In [37]:
%%time
# Loads reviews table
for df in pd.read_json(
    filePaths["reviews"], 
    lines=True,
    orient='records',
    chunksize=100000, # Dataframe size
    nrows=4234236234234
):
    new_reviews = pd.DataFrame(df)
    new_reviews.to_sql('Reviews', con=conn, if_exists='append')

Wall time: 5min 52s



INSTALLED VERSIONS
------------------
commit           : db08276bc116c438d3fdee492026f8223584c477
python           : 3.8.5.final.0
python-bits      : 64
OS               : Windows
OS-release       : 10
Version          : 10.0.19041
machine          : AMD64
processor        : AMD64 Family 21 Model 2 Stepping 0, AuthenticAMD
byteorder        : little
LC_ALL           : None
LANG             : None
LOCALE           : English_United States.1252

pandas           : 1.1.3
numpy            : 1.19.2
pytz             : 2020.1
dateutil         : 2.8.1
pip              : 20.2.4
setuptools       : 50.3.1.post20201107
Cython           : 0.29.21
pytest           : 6.1.1
hypothesis       : None
sphinx           : 3.2.1
blosc            : None
feather          : None
xlsxwriter       : 1.3.7
lxml.etree       : 4.6.1
html5lib         : 1.1
pymysql          : None
psycopg2         : None
jinja2           : 2.11.2
IPython          : 7.19.0
pandas_datareader: None
bs4              : 4.9.3
bottleneck     